# Predspracovanie dát

In [64]:
import matplotlib.pyplot as plt
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

from integration.integration import integration
from pyspark.sql import SparkSession
from collections import Counter

In [65]:
spark = SparkSession.builder.appName("preprocessing").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [66]:
df_train, df_test = integration()
df_train = df_train.drop("Accident_Index", "Date")
df_test = df_test.drop("Accident_Index","Date")

#### Zistenie, ktoré stĺpce sú kvantitatívne a kategorické

In [67]:
types = df_train.dtypes
print(types)

[('Location_Easting_OSGR', 'int'), ('Location_Northing_OSGR', 'int'), ('Longitude', 'double'), ('Latitude', 'double'), ('Police_Force', 'int'), ('Accident_Severity', 'int'), ('Number_of_Vehicles', 'int'), ('Number_of_Casualties', 'int'), ('Day_of_Week', 'int'), ('Time', 'timestamp'), ('Local_Authority_(District)', 'int'), ('Local_Authority_(Highway)', 'string'), ('1st_Road_Class', 'int'), ('1st_Road_Number', 'int'), ('Road_Type', 'int'), ('Speed_limit', 'int'), ('Junction_Detail', 'int'), ('Junction_Control', 'int'), ('2nd_Road_Class', 'int'), ('2nd_Road_Number', 'int'), ('Pedestrian_Crossing-Human_Control', 'int'), ('Pedestrian_Crossing-Physical_Facilities', 'int'), ('Light_Conditions', 'int'), ('Weather_Conditions', 'int'), ('Road_Surface_Conditions', 'int'), ('Special_Conditions_at_Site', 'int'), ('Carriageway_Hazards', 'int'), ('Urban_or_Rural_Area', 'int'), ('Did_Police_Officer_Attend_Scene_of_Accident', 'int'), ('LSOA_of_Accident_Location', 'string'), ('Vehicle_Reference', 'int')

In [68]:
counts = Counter()

for dtype in types:
    counts[dtype[1]] += 1

print(counts)

Counter({'int': 59, 'double': 2, 'string': 2, 'timestamp': 1})


#### Vypočítanie základných štatistík pre kvantitatívne stĺpce

In [69]:
def get_int(df):
    int_columns = []

    for column, data_type in df.dtypes:
        if data_type == "int":
            int_columns.append(column)
    return int_columns

In [70]:
def show_stats(df, columns, batch_size=4):
    for i in range(0, len(columns), batch_size):
        df.describe(columns[i:i+batch_size]).show()

show_stats(df_train, get_int(df_train))

[4464.958s][warning][gc,alloc] Executor task launch worker for task 2.0 in stage 930.0 (TID 2364): Retried waiting for GCLocker too often allocating 1048578 words


+-------+---------------------+----------------------+------------------+------------------+
|summary|Location_Easting_OSGR|Location_Northing_OSGR|      Police_Force| Accident_Severity|
+-------+---------------------+----------------------+------------------+------------------+
|  count|               300276|                300276|            300293|            300293|
|   mean|      438255.11356885|    300105.18741757586| 30.78688480916971|2.8231294102759636|
| stddev|    94748.83907367043|    157397.67714086492|24.743146328252084|0.4288013609691326|
|    min|                65860|                 10560|                 1|                 1|
|    max|               655370|               1191500|                98|                 3|
+-------+---------------------+----------------------+------------------+------------------+



+-------+------------------+--------------------+------------------+--------------------------+
|summary|Number_of_Vehicles|Number_of_Casualties|       Day_of_Week|Local_Authority_(District)|
+-------+------------------+--------------------+------------------+--------------------------+
|  count|            300293|              300293|            300293|                    300293|
|   mean|2.3571978034786025|   2.121068423173367| 4.110751832377045|          355.383961664108|
| stddev|  2.50916130396624|  3.2845832805738686|1.9431281459177299|        252.13048697950327|
|    min|                 1|                   1|                 1|                         1|
|    max|                67|                  93|                 7|                       941|
+-------+------------------+--------------------+------------------+--------------------------+



+-------+------------------+------------------+------------------+------------------+
|summary|    1st_Road_Class|   1st_Road_Number|         Road_Type|       Speed_limit|
+-------+------------------+------------------+------------------+------------------+
|  count|            300293|            300293|            300293|            300293|
|   mean| 3.912422200983706| 998.2143373305405|5.0935453040863425| 41.09957608069452|
| stddev|1.4508332728073978|1808.0544016265796| 1.647924197132397|15.104004572455908|
|    min|                 1|                -1|                 1|                10|
|    max|                 6|              9999|                 9|                70|
+-------+------------------+------------------+------------------+------------------+



+-------+------------------+------------------+-----------------+------------------+
|summary|   Junction_Detail|  Junction_Control|   2nd_Road_Class|   2nd_Road_Number|
+-------+------------------+------------------+-----------------+------------------+
|  count|            300293|            300293|           300293|            300293|
|   mean|2.3728924750160676|1.7908076445338386|2.622718478286207| 386.6823202672057|
| stddev|2.6134518815478947| 2.303394959580854|3.211294477117478|1312.3022275267308|
|    min|                -1|                -1|               -1|                -1|
|    max|                 9|                 4|                6|              9999|
+-------+------------------+------------------+-----------------+------------------+



...

+-------+------------------------+-------------------------+---------------------------+--------------------------+
|summary|Skidding_and_Overturning|Hit_Object_in_Carriageway|Vehicle_Leaving_Carriageway|Hit_Object_off_Carriageway|
+-------+------------------------+-------------------------+---------------------------+--------------------------+
|  count|                  300293|                   300293|                     300293|                    300293|
|   mean|      0.2268517747666446|       0.3010626288325069|        0.40768516082625966|        0.6090618162927541|
| stddev|       0.723789640107202|       1.5939520605809545|         1.4497338141550096|         2.167374561797518|
|    min|                      -1|                       -1|                         -1|                        -1|
|    max|                       5|                       12|                          8|                        11|
+-------+------------------------+-------------------------+------------

[4597.985s][warning][gc,alloc] Executor task launch worker for task 1.0 in stage 1134.0 (TID 2879): Retried waiting for GCLocker too often allocating 1048578 words


+-------+-------------------+----------------------------+-------------------------+------------------+
|summary|1st_Point_of_Impact|Was_Vehicle_Left_Hand_Drive?|Journey_Purpose_of_Driver|     Sex_of_Driver|
+-------+-------------------+----------------------------+-------------------------+------------------+
|  count|             300293|                      300293|                   300293|            300293|
|   mean| 1.7344160536542643|          0.9889507913937388|        8.837645233155618|1.3894696180064137|
| stddev| 1.1190713499544918|         0.18852904383972255|        5.980739624197323|0.5807067891130099|
|    min|                 -1|                          -1|                       -1|                -1|
|    max|                  4|                           2|                       15|                 3|
+-------+-------------------+----------------------------+-------------------------+------------------+



+-------+------------------+------------------+--------------------+------------------+
|summary|     Age_of_Driver|Age_Band_of_Driver|Engine_Capacity_(CC)|   Propulsion_Code|
+-------+------------------+------------------+--------------------+------------------+
|  count|            300293|            300293|              300293|            300293|
|   mean| 34.95248973502546| 5.968504094334533|    1467.86069272344|0.8100921433400046|
| stddev|19.064810411524313| 2.815962035850567|  1678.0161198258568|1.1314741218330875|
|    min|                -1|                -1|                  -1|                -1|
|    max|                99|                11|               99999|                12|
+-------+------------------+------------------+--------------------+------------------+



+-------+-----------------+------------------+---------------------+
|summary|   Age_of_Vehicle| Driver_IMD_Decile|Driver_Home_Area_Type|
+-------+-----------------+------------------+---------------------+
|  count|           300293|            300293|               300293|
|   mean|5.011478789049362| 3.630224480757127|   0.9206941220741076|
| stddev|5.294180187151661|3.7267864561304447|   1.1077424779630716|
|    min|               -1|                -1|                   -1|
|    max|              110|                10|                    3|
+-------+-----------------+------------------+---------------------+



In [74]:
def get_categorical(df):
    categorical_columns = []

    for column, data_type in df.dtypes:
        if data_type == "string":
            categorical_columns.append(column)
    return categorical_columns

In [75]:
def histogram(df, col, bins=10, xname=None, yname=None):


    # Calculating histogram in Spark
    vals = df.select(col).rdd.flatMap(lambda x: x).histogram(bins)

    # Preprocessing histogram points and locations
    width = vals[0][1] - vals[0][0]
    loc = [vals[0][0] + (i+1) * width for i in range(len(vals[1]))]

    # Making a bar plot
    plt.bar(loc, vals[1], width=width)
    plt.xlabel(col)
    plt.ylabel(yname)
    plt.show()

In [76]:
histogram(df_train, 'LSOA_of_Accident_Location', bins=15, yname='Number of countries')

TypeError: Can not generate buckets with non-number in RDD